In [1]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://sparkify-data-lake-tables/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1581600546771_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:

def create_spark_session():
    """returns: Spark Session object
    
    Description: Factory function that creates Spark Session objects.
    
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

spark = create_spark_session()

In [3]:
song_data = input_data + 'song_data/*/*/*/*.json'
df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|233.22077|        1|SOVIYJY12AF72A4B00|The Dead Next Doo...|1983|
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|287.92118|        1|SOVYXYL12AF72A3373|Rebel Yell (1999 ...|1983|
|ARQ846I1187B9A7083|           null|                    |            null|Yvonne S. Moriart...|196.04853|        1|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...|   0|
|AR4T2IF1187B9ADBB7|       63.9602

In [5]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [6]:
df.createOrReplaceTempView('song_data_table')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:

# extract columns to create songs table
songs_table = spark.sql("""
                                SELECT DISTINCT artist_id, song_id, title, year, artist_name, duration
                                FROM song_data_table
                                WHERE song_id IS NOT NULL
                            """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
songs_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+----+---------------+---------+
|         artist_id|           song_id|               title|year|    artist_name| duration|
+------------------+------------------+--------------------+----+---------------+---------+
|AR3793X1187FB50CB3|SODHVES12A6701CE6B|Driving Home For ...|1986|      Chris Rea|  241.162|
|AR74YRW1187FB3F834|SOHTHBB12AB018E6A8|         Rock N Roll|   0|   The Runaways|196.80608|
|AR5KAA01187FB5AEB7|SOLLGNV12A6701C582|Mimosa (Alternate...|   0| Herbie Hancock|606.61506|
|AR51T3E1187FB3B997|SOGBMLK12A8C139989|         Now Now Now|1990| Poverty Stinks|176.37832|
|ARRMAIV12086C12AE2|SOJZSHY12A8C137A17|Perjantaina (Albu...|2008|Haloo Helsinki!|222.87628|
+------------------+------------------+--------------------+----+---------------+---------+
only showing top 5 rows

In [9]:
songs_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)

In [10]:
# extract columns to create artists table
artists_table = spark.sql("""
                             SELECT artist_name, artist_id, artist_location, artist_latitude, artist_longitude
                             FROM song_data_table
                             WHERE artist_id IS NOT NULL
                          """)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
artists_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+--------------------+---------------+----------------+
|         artist_name|         artist_id|     artist_location|artist_latitude|artist_longitude|
+--------------------+------------------+--------------------+---------------+----------------+
|          Billy Idol|AR4T2IF1187B9ADBB7|<a href="http://b...|       63.96027|        10.22442|
|          Billy Idol|AR4T2IF1187B9ADBB7|<a href="http://b...|       63.96027|        10.22442|
|Yvonne S. Moriart...|ARQ846I1187B9A7083|                    |           null|            null|
|          Billy Idol|AR4T2IF1187B9ADBB7|<a href="http://b...|       63.96027|        10.22442|
|Russell Watson / ...|AR3TZ691187FB3DBB1|                    |           null|            null|
+--------------------+------------------+--------------------+---------------+----------------+
only showing top 5 rows

In [25]:
artists_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_name: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)

In [27]:
log_data = input_data + 'log_data/*/*/*.json'

# read log data file
log_df = spark.read.json(log_data)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
log_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|The Big Gundown|

In [29]:
log_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [30]:
log_df.createOrReplaceTempView('log_data_table')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# filter by actions for song plays
log_df = spark.sql("""
                    SELECT * FROM log_data_table
                    WHERE page = 'NextSong'
                   """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# extract columns for users table    
users_table = spark.sql("""
                            SELECT DISTINCT firstName, lastName, gender, userId 
                            FROM log_data_table
                            WHERE userId IS NOT NULL
                        """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
users_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+------+------+
|firstName|lastName|gender|userId|
+---------+--------+------+------+
| Anabelle| Simpson|     F|    69|
|   Sylvie|    Cruz|     F|    10|
|   Harper| Barrett|     M|    42|
|  Cecilia|   Owens|     F|     6|
|   Jaleah|   Hayes|     F|    70|
+---------+--------+------+------+
only showing top 5 rows

In [34]:
users_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- userId: string (nullable = true)

In [35]:
ts_df = spark.sql("""SELECT *,to_timestamp(ts/1000) as start_time
                            FROM log_data_table""")
ts_df.createOrReplaceTempView('timestamp_df')

# extract columns to create time table
time_table = spark.sql("""SELECT 
                            start_time,
                            hour(start_time) as hour,
                            dayofmonth(start_time) as day,
                            weekofyear(start_time) as week,
                            month(start_time) as month,
                            year(start_time) as year,
                            dayofweek(start_time) as weekday
                            FROM timestamp_df                          
                        """)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [37]:
time_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:41:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:45:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 01:57:...|   1| 15|  46|   11|2018|      5|
|2018-11-15 03:29:...|   3| 15|  46|   11|2018|      5|
+--------------------+----+---+----+-----+----+-------+
only showing top 5 rows

In [38]:
songs_table.createOrReplaceTempView('song_table')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# extract columns from joined song and log datasets to create songplays table 
songplays_df = spark.sql("""SELECT 
                                monotonically_increasing_id() as songplay_id, 
                                log_data_table.userId as user_id,
                                to_timestamp(log_data_table.ts/1000) as start_time,
                                month(to_timestamp(log_data_table.ts/1000)) as month,
                                year(to_timestamp(log_data_table.ts/1000)) as year,
                                log_data_table.level as level,
                                song_table.song_id as song_id,
                                song_table.artist_id as artist_id,
                                log_data_table.sessionId as session_id,
                                log_data_table.location as location,
                                log_data_table.userAgent as user_agent
                            FROM song_table 
                            JOIN log_data_table ON song_table.artist_name = log_data_table.artist 
                                                    AND song_table.title = log_data_table.song
                        """)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
songplays_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------+--------------------+-----+----+-----+------------------+------------------+----------+--------------------+--------------------+
|songplay_id|user_id|          start_time|month|year|level|           song_id|         artist_id|session_id|            location|          user_agent|
+-----------+-------+--------------------+-----+----+-----+------------------+------------------+----------+--------------------+--------------------+
|          0|     49|2018-11-15 07:31:...|   11|2018| paid|SOCUITT12AB0187A32|ARKS2FE1187B99325D|       606|San Francisco-Oak...|Mozilla/5.0 (Wind...|
|          1|     80|2018-11-15 10:44:...|   11|2018| paid|SOSDZFY12A8C143718|AR748W61187B9B6AB8|       611|Portland-South Po...|"Mozilla/5.0 (Mac...|
|          2|     80|2018-11-15 10:48:...|   11|2018| paid|SOTNWCI12AAF3B2028|ARS54I31187FB46721|       611|Portland-South Po...|"Mozilla/5.0 (Mac...|
|          3|     80|2018-11-15 10:52:...|   11|2018| paid|SOBONKR12A58A7A7E0|AR5E44Z1187B9A1D

In [41]:
songplays_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- songplay_id: long (nullable = false)
 |-- user_id: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)

In [ ]:
# Time Consuming operations of writing data to S3 Bucket

# Some insights about the cause:
# https://stackoverflow.com/questions/42822483/extremely-slow-s3-write-times-from-emr-spark

In [42]:

# write artists table to parquet files
artists_table.limit(100).write.mode('overwrite').parquet(output_data + "artists_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# write songs table to parquet files partitioned by year and artist
songs_table.limit(10).write.mode('overwrite').partitionBy(["year","artist_id"]).parquet(output_data + "song_table_v2/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# write users table to parquet files
users_table.limit(10).write.mode('overwrite').parquet(output_data + "users_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# write time table to parquet files partitioned by year and month
time_table.limit(10).write.mode('overwrite').partitionBy(["year","month"]).parquet(output_data + "time_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:

# write songplays table to parquet files partitioned by year and month
songplays_df.limit(10).write.mode('overwrite').partitionBy(["year","month"]).parquet(output_data + "songplay/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…